<a href="https://colab.research.google.com/github/mwele/applied_ai_code/blob/master/day_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!pip install -q kaggle
import numpy as np
import pylab
import scipy.stats as stats
import matplotlib.pyplot as plt 
import pandas as pd
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kyalomwele","key":"dd7a764fb631185569ccc890cabe77fc"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import sqlite3 
import numpy as np
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns 
data=pd.read_csv('Reviews.csv')
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1.0,1.0,5.0,1.303862e+09,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0.0,0.0,1.0,1.346976e+09,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1.0,1.0,4.0,1.219018e+09,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3.0,3.0,2.0,1.307923e+09,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0.0,0.0,5.0,1.350778e+09,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
29936,29937,B000FDLAZ6,AZ5AK5C6ZP84S,Mark D. Griffis,0.0,0.0,5.0,1.326586e+09,Better Than Homemade Without the Time and Hassle,This soup mix makes a tasty addition to any mu...
29937,29938,B000FDLAZ6,AK6J92Q930IFO,"Mrs. M. ""Health Concerns""",0.0,0.0,3.0,1.324166e+09,green leafy vegetable,If you lack dark green vegetables this will su...
29938,29939,B001I8EVZI,A3EUSH8DPW75LR,barry,11.0,11.0,5.0,1.306627e+09,Great coffee when the price is right,I love my coffee and have tried all the name b...
29939,29940,B001I8EVZI,A3S3DNSUREE2NK,"Mistress of The Dark Path ""Suzie""",2.0,2.0,5.0,1.295827e+09,The best flavor for the price,Folgers classic roast has been my trusted bran...


In [ ]:
filtered_data=data[data['Score']!=3]
filtered_data.dropna(subset=['Score'])

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1.0,1.0,5.0,1.303862e+09,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0.0,0.0,1.0,1.346976e+09,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1.0,1.0,4.0,1.219018e+09,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3.0,3.0,2.0,1.307923e+09,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0.0,0.0,5.0,1.350778e+09,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
29933,29934,B001EQ5F9K,A155EX606XWQCW,JD,3.0,10.0,1.0,1.294963e+09,Don't be fooled by the price!!!!!,Although the cereal is a great tasting cereal ...
29935,29936,B000FDLAZ6,A2YSFKX18XTHED,"Billie Vanderburg ""Smart Shopper""",3.0,4.0,5.0,1.219968e+09,Delicious and So Easy!,This tastes just like homemade soup. I wish I ...
29936,29937,B000FDLAZ6,AZ5AK5C6ZP84S,Mark D. Griffis,0.0,0.0,5.0,1.326586e+09,Better Than Homemade Without the Time and Hassle,This soup mix makes a tasty addition to any mu...
29938,29939,B001I8EVZI,A3EUSH8DPW75LR,barry,11.0,11.0,5.0,1.306627e+09,Great coffee when the price is right,I love my coffee and have tried all the name b...


In [ ]:
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True,inplace=False)
sorted_data
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first', inplace=False)
final['Score']

24750    5.0
24749    2.0
2774     5.0
2773     4.0
1243     5.0
        ... 
12618    4.0
12619    4.0
12620    4.0
1477     1.0
5702     5.0
Name: Score, Length: 26272, dtype: float64

In [ ]:
def partition(x):
    
    if x < 3.0:
        return 'negative'
    else:
        return 'positive'

In [ ]:
#changing from numerals 1,2 3,4,5 to whether it's positive or negative
actualScore=final['Score']
#print("actual Score:"+ str(actualScore))
positiveNegative=actualScore.map(partition)
#print("positive negative:"+ str(positiveNegative))
filtered_data['Score']=positiveNegative

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re 
import nltk
import string
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,'',sentence)
    return cleantext 
def cleanpunc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r'',cleaned)
    return cleaned
print(stop)
print("*********************************************")
print(sno.stem('taste'))

{'very', 'won', 'ain', 'shouldn', 'that', "wouldn't", 'or', 'she', "you've", 'who', 'couldn', 'on', 't', 'during', 'no', 'while', 'don', 'i', 'me', 'into', 'were', 'can', 'our', 'and', 'up', 'to', 'its', 'y', 'over', 'in', 'mustn', 'down', 'through', 'it', 'between', "she's", 'did', 'not', "needn't", "haven't", 'isn', 'what', 'haven', 'against', 'was', 'the', 'off', 'has', "you'll", 'd', 'hasn', 'weren', 'each', 'am', 'but', "you'd", 'once', 'some', "aren't", 'didn', 'shan', 'wasn', 'here', 'needn', 'any', 'same', 'so', 're', 'under', 'more', 'm', 'hers', 'him', "hasn't", 'ma', 'above', 'll', 'had', "doesn't", 'as', 'whom', 'a', 'from', 'they', 'herself', "that'll", 'if', 'these', 'how', 'her', 'which', 'my', 'be', 'theirs', 'those', 'o', 's', 'then', 'because', 'most', 'have', 'itself', "isn't", "mustn't", 'again', 'themselves', 'yourself', 'of', 'there', 'doing', 'such', 'why', 'been', 'wouldn', 'when', 'at', 'own', "couldn't", "shouldn't", 'for', 'will', "don't", 'does', "should've"

In [ ]:
i=0
str1=''
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if ((cleaned_words.isalpha()))&(len(cleaned_words)>2):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf-8')
                    filtered_sentence.append(s)
                    if(final['Score'].values)[i]=='positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i]=='negative':
                        all_negative_words.append(s)
                else:
                    continue
            str1=b" ".join(filtered_sentence)
            final_string.append(str1)
            i+=1
            
            

IndexError: ignored